In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
import os

In [5]:
dataset_folder = os.path.join(os.environ['HOME'], 'hse/data/hotdogs_dataset')

In [6]:
test_folder = os.path.join(dataset_folder, 'test')
train_folder = os.path.join(dataset_folder, 'train')

In [7]:
os.listdir(train_folder)

['hotdog', 'not_hotdog']

In [8]:
def get_hotdog_class_dirs(root_dir):
    return os.path.join(root_dir, 'hotdog'), os.path.join(root_dir, 'not_hotdog')

In [9]:
hotdogs_folder, not_hotdogs_folder = get_hotdog_class_dirs(train_folder)

In [10]:
os.listdir(hotdogs_folder)[:10]

['00651.jpg',
 '00337.jpg',
 '00287.jpg',
 '00506.jpg',
 '00064.jpg',
 '00132.jpg',
 '00351.jpg',
 '00334.jpg',
 '00601.jpg',
 '00107.jpg']

In [11]:
def path_join(left, right):
    return os.path.join(left, right)

In [14]:
from PIL import Image

def fix_size(orig_size, needed_size):
    """calculate size keeping the aspect ratio"""
    # 1. find small side
    smaller_side_index = np.argmin(orig_size)
    bigger_side_index = 1 - smaller_side_index
    # 2. scale by small side
    aspect_ratio = orig_size[smaller_side_index] / needed_size[smaller_side_index]
    # 3. return new size with correct aspect ratio
    size = [0, 0]
    size[smaller_side_index] = needed_size[smaller_side_index]
    size[bigger_side_index] = int(needed_size[bigger_side_index] / aspect_ratio)
    return size

def square(size):
    return size[0] == size[1]

def roi(size_with_ratio, out_size):
    """calculate region of interest"""
    # 1. find small side
    smaller_side_index = np.argmin(size_with_ratio)
    bigger_side_index = 1 - smaller_side_index
    # 2. calculate region of interest
    #    a. small side is untouched
    #    b. bigger side is cropped
    box = [0, 0, 0, 0]
    # 2.a
    box[smaller_side_index] = 0
    box[smaller_side_index+2] = out_size[smaller_side_index]
    # 2.b
    bigger_side_center = int(size_with_ratio[bigger_side_index] / 2)
    box[bigger_side_index] = bigger_side_center - out_size[bigger_side_index] / 2
    # + whole number to eliminate problems with non-integer centers:
    box[bigger_side_index+2] = box[bigger_side_index] + out_size[bigger_side_index]
    return box

def preprocess_images(image_dir, out_size):
    """preprocess images"""
    if not square(out_size):
        raise NotImplementedError

    image_files = os.listdir(image_dir)
    good_images = []
    bad_files = []
    # 1. load image - if fails, skip
    for file in image_files:
        fp = open(path_join(image_dir, file), "rb")
        try:
            img = Image.open(fp)
            size_with_ratio = fix_size(img.size, out_size)
            img = img.resize(size_with_ratio, Image.HAMMING)
            if not square(size_with_ratio):  # crop center if image is not a square
                img = img.crop(box=roi(size_with_ratio, out_size))
            # 2. create tuple of image name and resized image object
            good_images.append((file, img))
        except IOError:  # bad image/not an image
            bad_files.append(file)
    # 3. return tuples list and list of bad images (for debug primarily)
    return good_images, bad_files

In [15]:
hotdog_images, hotdog_errors = preprocess_images(hotdogs_folder, (224, 224))
not_hotdog_images, not_hotdog_errors = preprocess_images(not_hotdogs_folder, (224, 224))

(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)

KeyboardInterrupt: 

In [190]:
# def rgb_to_int(rgb):
#     r, g, b = rgb
#     r = r << 8 * 2
#     g = g << 8 * 1
#     b = b
#     return int(r|g|b)

# from PIL import ImageStat

# def image_to_array(img, rgb_aware):
#     img.load()
#     arr = np.asarray(img, dtype="int32")
#     if rgb_aware:
#         new_arr = []
#         for row in arr:
#             new_row = []
#             for rgb in row:
#                 new_row.append(rgb_to_int(rgb))
#             new_arr.append(new_row)
#         arr = np.array(new_arr)
#     return arr.reshape(-1)

# # TODO: add separate logic for grayscale
# def image_to_array_multi(images, concat_rgb=False, debug=False):
#     names = []
#     data = []
#     for name, img in images:
#         arr = image_to_array(img, True)
#         data.append(arr)
#     return names, data

In [135]:
hotdog_arrays = image_to_array_multi(hotdog_images, True)

In [191]:
not_hotdog_arrays = image_to_array_multi(not_hotdog_images, True)

KeyboardInterrupt: 

In [ ]:
hotdog_arrays[:5]

In [109]:
pd.Series(hotdog_arrays[0]).head()

0    00651.jpg
1    00337.jpg
2    00287.jpg
3    00506.jpg
4    00064.jpg
dtype: object

In [111]:
pd.DataFrame(hotdog_arrays[1]).head()

,0,1,2,3,4,5,6,7,8,9,...,150518,150519,150520,150521,150522,150523,150524,150525,150526,150527
0,19,19,27,17,17,25,15,15,23,14,...,89,64,78,91,62,78,92,62,78,93
1,36,13,0,39,14,1,41,15,2,42,...,164,132,133,163,130,131,161,134,135,165
2,8,8,8,9,9,9,7,7,7,9,...,105,120,116,108,116,112,102,116,112,101
3,246,224,244,244,223,242,243,223,242,242,...,221,241,192,213,242,201,219,241,203,220
4,219,141,144,194,114,117,196,116,119,197,...,233,204,218,231,205,219,232,215,227,240


In [107]:
hotdog_labels = pd.Series([1]*len(hotdog_arrays))
not_hotdog_labels = pd.Series([0]*len(not_hotdog_arrays))

train_hotdogs_pd = pd.DataFrame(
    hotdog_arrays[0],
    hotdog_labels,
    hotdog_arrays[1],
#     columns=['name', 'hotdog']
)
train_not_hotdogs_pd = pd.DataFrame(
    not_hotdog_arrays[0],
    not_hotdogs_labels,
    not_hotdog_arrays[1],
#     columns=['name', 'hotdog']
)

ValueError: Shape of passed values is (1, 708), indices imply (150528, 2)

In [51]:
train_hotdogs_pd.head()

,name,image
0,00651.jpg,"[19, 19, 27, 17, 17, 25, 15, 15, 23, 14, 15, 2..."
1,00337.jpg,"[36, 13, 0, 39, 14, 1, 41, 15, 2, 42, 14, 2, 4..."
2,00287.jpg,"[8, 8, 8, 9, 9, 9, 7, 7, 7, 9, 9, 9, 9, 9, 9, ..."
3,00506.jpg,"[246, 224, 244, 244, 223, 242, 243, 223, 242, ..."
4,00064.jpg,"[219, 141, 144, 194, 114, 117, 196, 116, 119, ..."
